In [5]:
import pretty_midi
import glob
import random
import json

In [6]:
import fnmatch
import os

file_paths = []

for root, dirnames, filenames in os.walk('data'):
    for filename in fnmatch.filter(filenames, '*.mid'):
        file_paths.append(os.path.join(root, filename))

In [7]:
from multiprocessing.dummy import Pool as ThreadPool 

In [78]:
random.shuffle(file_paths)

In [79]:
len(file_paths)

17256

In [9]:
id_and_fp = list(enumerate(file_paths))

In [10]:
pool = ThreadPool(4)

In [11]:
def worker(id_and_fp):
    outfile, fpath = id_and_fp
    try:
        pm = pretty_midi.PrettyMIDI(midi_file=fpath)
    except:
        return "Failed"

    chro = pm.get_chroma()
    chro = chro.astype(bool)
    rows = []

    last_added = []
    for row in chro.T:
        if list(row) != last_added:
            rows.append(list(row))
            last_added = list(row)
    
    def row_to_str(row):
        return "".join(['0' if j == False else '1' for j in row])
    
    rows_str = [row_to_str(r) for r in rows]
    
    obj = {
        "path" : fpath,
        "rows" : rows_str
    }
    
    with open('{}.json'.format(outfile), 'w') as jf:
        json.dump(obj, jf)
    
    return "Success"

In [12]:
results = pool.map(worker, id_and_fp)